<a href="https://colab.research.google.com/github/MuhammadHammad-git/The-Tsiolkovsky-rocket-equation/blob/main/The_Tsiolkovsky_rocket_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement
The Tsiolkovsky rocket equation is an ideal rocket equation which has the direct relation between change in velocity ($\Delta v$), exhaust gases velocity ($v_0$), wet mass of the rocket ($m_0$), i.e. weight of the rocket at the time of take off including the weight of the fuel and dry mass of the rocket ($m_f$), i.e. when rocket has burnt all the fuel.

$$\Delta v = v_0 \log (m_0/m_f)$$

This problem optimizes the design of a simple rocket for profit. Potential revenue increases with greater change in velocity, as greater velocities allow the payload to reach higher orbits that have less drag, allowing it to remain in orbit longer. Wet mass, dry mass, and exhaust velocity are design variables, where wet mass is the total initial mass of the rocket, including propellant, and dry mass is the mass of the rocket at full ascent.



![Rocket](https://drive.google.com/uc?id=1zpdoyHLf-SvXatv4xmW1_MFiiYBfjzNA)

The rocket must have a dry mass of at least $20,000$ kilograms and the change in velocity should be between $9,400$ meters per second and $20,200$ meters per second. Varying designs allow for exhaust velocities ranging from $2,500 m/s$ to $4,500 m/s$. An appropriate guess value for the wet mass is $150,650 km$.

The overall profit from the rocket is:

$$Profit = R - C_f - C_d - C_e$$

where $C_f$ is the cost of the fuel, given by
$$C_f = 4.154 (m_0-m_f)$$

$C_d$ is the cost of the rocket
 $$C_d = 154.36 * m_f$$

$C_e$ is the cost in relation to adusting the exhaust velocity
 $$C_e= 75 v_0$$

 R is the revenue and is given by
 $$R=550 \Delta v$$

# Code

In [ ]:
import math
from scipy.optimize import Bounds,minimize
import numpy as np

## Objective function
Profit is multiplied by -1 because it has to be maximized using a minimization function.

In [ ]:
def objective_function(x):
  v0=x[0]
  m0 = x[1]
  mf = x[2]

  Ce= 75 * v0
  Cd= 154.36 * mf
  Cf= 4.154*(m0-mf)
  dv=v0*math.log((m0/mf))
  R= 550*dv
  profit = R-Cf-Cd-Ce
  return -1*profit



## bounds
Note: Bounds are implemented using the Bounds class which was the second method mentioned in the scipy.optimize.minimize documentation. It is slightly different from what we covered in last lecture.

* bounds for exhaust velocity are provided
* Lower bound of 21000 is assumed for the rocket. Because rocket's wet mass would be greater than rocket's dry mass. If this bound is skiped, it might not effect the optimization alogrithm results, but on the other hand the algo. will look for broader range of value for m0, which will increase the computational efforts. Upper bound limit can also be provided, say you can't built a rocket big enough to hold 3,000 tons of fuel.
* Lower bound on the rocket's dry mass is provided, i.e. 20,000. You can think of this limit as the weight below which rocket is safe enough or strong enough and it should include the payload of the rocket as well.


In [ ]:

lb = np.array([[2500],
               [21000],
               [20000]]) # in order of v0,m0 and mf

up = np.array([[4500],
               [np.inf],
               [np.inf]])
mybounds=Bounds(lb,up)

x0=np.array([[3500],
             [150650],
             [21000]])

## Constraints
Rocket's change in velocity ($d_v$) has an upper and lower limit but it can't be treated as a bound, because $d_v$ is not a design variable and it is being calculated by the design variables. Currently, the constraint on $d_v$ is implemented in the form of two constraints for the sake of simplicity but it can be implemented as one constraint as well.

Physical reasoning behind this contraint could be that if rocket moves faster than $20200 m/s$ then it can burn due to air friction at the time of lift off and might not be able to exit the earth's gravity if change in velocity is less $9400 m/s$

In [ ]:
def constraint1(x):
  v0=x[0]
  m0 = x[1]
  mf = x[2]
  dv=v0*math.log((m0/mf),10)-9400
  return dv

def constraint2(x):
  v0=x[0]
  m0 = x[1]
  mf = x[2]
  dv=v0*math.log((m0/mf),10)+20200
  return dv

con1={'fun':constraint1,'type':'ineq'}
con2={'fun':constraint2,'type':'ineq'}
myconstraints = (con1,con2)

In [ ]:
sol = minimize(objective_function, x0, method='SLSQP', bounds=mybounds, constraints=myconstraints)
sol


     fun: 1632211.5616472778
     jac: array([-2570.375  ,     3.125  ,   273.84375])
 message: 'Optimization terminated successfully.'
    nfev: 95
     nit: 22
    njev: 18
  status: 0
 success: True
       x: array([   4500.        , 2454250.47962062,   20000.        ])

In [ ]:
objective_function(x0)

array([249535.20638319])

### Whats the optimized value of the objective function?

In [ ]:
objective_function(sol.x)

1632211.5616472778

### For what exhaust velocity the rocket should be designed?
4500 m/s

In [ ]:
sol.x[0]

4500.0

### For what wet mass the rocket should be designed?
2454250.47962062 kg

In [ ]:
sol.x[1]

2454250.4796206155